In [13]:
from csv import reader
from math import sqrt
from math import exp
from math import pi
import pandas as pd

# Load a CSV file
def insert_data(filename):
    dataset = list()
    with open(filename, 'r') as file:  
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset

def str_2_int(dataset, column):
    class_values = [row[column] for row in dataset]
    #print("class value-",class_values)
    unique = set(class_values)
    #unique=unique.sort()
    #print(unique)
    lookup = dict()
    for i, value in enumerate(unique): #integer value to each unique variable
        lookup[value] = i
        print('[%s] => %d' % (value, i)) #assign integer to unique variable value
    for row in dataset:
        row[column] = lookup[row[column]]
    return lookup

#----------crate model-------------------
#-----step-3a->sort each row based on the outcome value
#-----step 3b->find SD & means for all column for each class 

def create_model(dataset):
    separated = find_class(dataset) #put each row in 1[yes] or 0[No] bracket
    #print("seprated",separated)
    summaries = dict()
    for class_value, rows in separated.items(): #for each class valu elike zero there is one dataset
        #print("class valuer and row-",class_value,"data-\n", rows)
        summaries[class_value] = calculate_dataset(rows) #
    return summaries
#----sorting all rows based on Y value---------
def find_class(dataset):
    separated = dict()
    for i in range(len(dataset)):
        #print(dataset[i])
        vector = dataset[i]
        print(type(vector),vector)
        class_value = vector[-1]
        #print(class_value)
        
        if (class_value not in separated):
            separated[class_value] = list()
        separated[class_value].append(vector)
    print("claases-",separated)
    #all teh rows when out come is 0 or 1 , like sortinbgbased on 0 and 1
    return separated
    
# Calculate the mean, stdev for each column in given class
def calculate_dataset(dataset):
    #print(dataset)
    summaries = [(mean(column), stdev(column), len(column)) for column in zip(*dataset)]
    #print(summaries[-1])
    del(summaries[-1]) #deleting the value for last column of outcome
    return summaries

def mean(numbers):return sum(numbers)/float(len(numbers))

def stdev(numbers):
    avg = mean(numbers)
    variance = sum([(x-avg)**2 for x in numbers]) / float(len(numbers)-1)
    return sqrt(variance)

#####------------------------------------------------------

#----5c-------------
def calculate_probability(x, mean, stdev): #applying gaussian distribution
    #print(x)
    exponent = exp(-((x-mean)**2 / (2 * stdev**2 )))
    return (1 / (sqrt(2 * pi) * stdev)) * exponent


#----5b-----
def calculate_class_probabilities(summaries, row):
    total_rows = sum([summaries[label][0][2] for label in summaries]) #to get 5 and 9 row of yes and no
    probabilities = dict()
    print("total rows, summaries", total_rows, summaries)
    #summaries have standard deviation and means and noof rows
    for class_value, class_summaries in summaries.items():
        #print(class_value, class_summaries)
        
        probabilities[class_value] = summaries[class_value][0][2]/float(total_rows) #9/14, 5/14   p[yes], p[no]
        
        #print("that-",probabilities[class_value])
        
        for i in range(len(class_summaries)):
            #print(class_summaries[i])
            
            mean, stdev, _ = class_summaries[i] #storing value from 
            #print( mean, stdev, row[i])
            probabilities[class_value] *= calculate_probability(row[i], mean, stdev)
            print("prob-",class_value,"-",probabilities[class_value])
            
    return probabilities


#----5a------------------
def predict(summaries, row):
    probabilities = calculate_class_probabilities(summaries, row)
    print("prob-",probabilities)
    
    best_label, best_prob = None, -1 #variable assignment
    for class_value, probability in probabilities.items():
        print(class_value, probability)
        if best_label is None or probability > best_prob:
            best_prob = probability
            best_label = class_value
    print(best_prob, best_label)
    return best_label,best_prob


#--------------naive bayes----------
#--------------------------------------------------------
#---step-1----insert the data file
filename = 'play_tennis1.csv'                      
dataset = insert_data(filename) #load csv
print(dataset)

#--step2-covert value of column to integer-----------
for i in range(len(dataset[0])):    
     str_2_int(dataset, i)

#-----------------creating the model --------------------        
#-step3---pass the  value to find out the  meand and standard deviation of each column        
model = create_model(dataset)  #of class 1[yes] and zero[no]
print("model data-",model)

#standar deviation  and mean of each column , dependednt variable for 0 and 1

#step4--finding out the label and probability of given dataset

data_input=["Sunny","Hot","High","Weak","yes"] 
row = [1,2,1,0,1]
#print(row)

#-step5----assigning a label to given data set based on model crated
label = predict(model, row) #passing model and input data
new_label, prob=label[0], label[1]
print("label-",new_label, prob)

#--step-6 define outcome----
output=  "happen" if label==0  else "Not happen"
print(f"play will {output} with following conditions",data_input)
 

[['Sunny', 'Hot', 'High', 'Weak', 'No'], ['Sunny', 'Hot', 'High', 'Strong', 'No'], ['Overcast', 'Hot', 'High', 'Weak', 'Yes'], ['Rain', 'Mild', 'High', 'Weak', 'Yes'], ['Rain', 'Cool', 'Normal', 'Weak', 'Yes'], ['Rain', 'Cool', 'Normal', 'Strong', 'No'], ['Overcast', 'Cool', 'Normal', 'Strong', 'Yes'], ['Sunny', 'Mild', 'High', 'Weak', 'No'], ['Sunny', 'Cool', 'Normal', 'Weak', 'Yes'], ['Rain', 'Mild', 'Normal', 'Weak', 'Yes'], ['Sunny', 'Mild', 'Normal', 'Strong', 'Yes'], ['Overcast', 'Mild', 'High', 'Strong', 'Yes'], ['Overcast', 'Hot', 'Normal', 'Weak', 'Yes'], ['Rain', 'Mild', 'High', 'Strong', 'No']]
[Rain] => 0
[Overcast] => 1
[Sunny] => 2
[Hot] => 0
[Mild] => 1
[Cool] => 2
[Normal] => 0
[High] => 1
[Strong] => 0
[Weak] => 1
[No] => 0
[Yes] => 1
<class 'list'> [2, 0, 1, 1, 0]
<class 'list'> [2, 0, 1, 0, 0]
<class 'list'> [1, 0, 1, 1, 1]
<class 'list'> [0, 1, 1, 1, 1]
<class 'list'> [0, 2, 0, 1, 1]
<class 'list'> [0, 2, 0, 0, 0]
<class 'list'> [1, 2, 0, 0, 1]
<class 'list'> [2, 1,